References

- [Dicoding - Mendapatkan Prediksi Model dari Input Text](https://www.dicoding.com/academies/443/discussions/209385)  

In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [50]:
df   = pd.read_csv('data/disaster_tweets.csv')
text = df['text'][2]

In [54]:
def prepare_json(text):
    feature_spec = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]
    
    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [55]:
def predict(text):
    response = requests.post(
        'http://localhost:8080/v1/models/disaster-tweets-model:predict',
        data = prepare_json(text)
    )
    
    predict = response.json()['predictions'][0][0]
    return predict

In [56]:
predict = predict(text)

print(f'{text}\n')
print(f'Real Disaster ({predict})' if predict > 0.6 else f'Fake Disaster ({predict})')

All residents asked to  ' shelter in place '  are being notified by officers .  No other evacuation or shelter in place orders are expected

Real Disaster (0.877941132)
